In [117]:
import jsonlines
from collections import Counter

def load_exp(dataset, method):
    res = []
    if method == 'ours':
        ori = open('explanations/%s/%s/train.txt.explain_inj_bs' % (method, dataset)).readlines()
        for i in range(0, len(ori), 2):
            o = ori[i].strip()
            n = ori[i+1].strip()
            res.append((method, o, n))
    else:
        ori = {}
        with jsonlines.open('explanations/%s/%s/test.output.jsonl' % (method, dataset)) as reader:
            for row in reader:
                ins = row['left'] + '\t' + row['right'] + '\t' + str(row['match'])
                if row['row_id'] not in ori:
                    ori[row['row_id']] = ins
                else:
                    o = ori[row['row_id']]
                    res.append((method, o, ins))
                
    return res

def clean(s):
    left, right, _ = s.split('\t')
    left = left.replace(' ', '').replace('\t', '').replace('\n', '') # [:50]
    right = right.replace(' ', '').replace('\t', '').replace('\n', '') # [:50]
    return left + right

all_exp = []
# for dataset in ['AB', 'AG', 'DS', 'DA', 'WA']:
for dataset in ['AB']:
    for method in ['lime', 'shap', 'ours']:
        res = load_exp(dataset, method)
        print(dataset, method, len(res))
        all_exp += load_exp(dataset, method)
        
indices = {}
for m, o, n in all_exp:
    co = clean(o)
    if co not in indices:
        indices[co] = [(m, o, n)]
    else:
        indices[co].append((m, o, n))

def count_methods(ll):
    cnt = Counter()
    for m, _, _ in ll:
        cnt[m] += 1
    return len(cnt)
        
num = 0
for key in indices:
    if count_methods(indices[key]) >= 3:
        num += 1
        # print(count_methods(indices[key]))

print(num)

AB lime 9580
AB shap 7664
AB ours 958
958


In [131]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

def to_dict(ins):
    left, right, lbl = ins.strip().split('\t')
    res = []
    for elem in [left, right]:
        tpl = {}
        for attr_val in elem.split('COL')[1:]:
            attr, val = attr_val.split('VAL')
            attr = attr.strip()
            val = val.strip()
            tpl[attr] = val
            
        res.append(tpl)
    return res

def show_instance(ori, new_list):
    dfs = []
    for lr in range(2):
        kv = dict([(key, [ori[lr][key]]) for key in ori[lr]])
        for item in new_list:
            for attr in item[lr]:
                if attr in kv:
                    kv[attr].append(item[lr][attr])
        
        df = pd.DataFrame(kv)
        # for attr in df:
        #     df[attr] = df[attr].str.wrap(50)
        dfs.append(df)

    for df in dfs:
        display(df)
    
import random
keys = [k for k in indices.keys() if count_methods(indices[k]) >= 3]

while True:
    try:
        key = random.choice(keys)
        # num += 1
        _, o, _ = indices[key][0]
        label = o.split('\t')[2].strip()
        seen = set([])
        n_list = []
        m_list = []
        for m, _, n in indices[key]:
            if m not in seen:
                n_list.append(to_dict(n))
                m_list.append(m)
                seen.add(m)

        show_instance(to_dict(o), n_list)
        print('rows =', ['original'] + m_list, ', ori_label =', label)
        break
    except:
        pass

,name,description,price
0,logitech harmony 890 advanced universal remote control h890,logitech harmony 890 advanced universal remote control h890 quick and simple set up advanced online setup and update wizard uses both radio frequency ( rf ) and infrared ( ir ) wireless signals integrate up to 15 devices 8 on-screen activity buttons rechargeable lithium ion battery and charging station included fully backlit comfort-grip shape,
1,logitech harmony 890 advanced universal remote control h890,logitech harmony 890 advanced universal remote h890 quick and simple set up advanced setup update wizard uses both radio frequency ( rf ) and infrared ( ir ) wireless signals integrate up to 15 devices 8 on-screen buttons rechargeable lithium ion charging station included fully backlit comfort-grip shape,
2,logitech 890 advanced universal remote control h90,logitech 890 advanced universal remote control h8quick simple set up advanced online setup and update wizard uses both radio frequency ( rf ) and infrared ir ) wireless signals integrate up to 15 devices 8 on-screen activity buttons able lithium ion battery and charging station included fully backlit comfort-grip shape,
3,universal remote control mrf-260 universal remote control h890,logitech harmony 890 advanced universal remote control h890 quick and simple set up advanced online setup and update wizard uses both radio frequency ( rf ) and infrared ( ir ) wireless signals integrate up to 15 devices 8 on-screen activity buttons rechargeable lithium ion battery and charging station included fully backlit comfort-grip shape,


,name,description,price
0,universal remote control mrf-260 barrier-free addressable rf base station,,86.25
1,universal remote control mrf-260 barrier-free addressable rf base station,,
2,universal remote mrf260 -free addressable rf base station,,86.25
3,universal remote control mrf-260 barrier-free addressable rf base station,,86.25


rows = ['original', 'lime', 'shap', 'ours'] , ori_label = 0
